# Introduction to AVISO-data

We will use the satellite-altimetry-derived ocean-surface current product AVISO-data as a dataset to validate the methods. The data is freely available from the [AVISO]((http://aviso.oceanobs.com)).

The sea surface height $ h $ acts as a stream function for the geostrophic ocean surface velocity field. Particle trajectories on the ocean surface are approximately trajectories of the 2D system of ODEs:

\begin{align}
\dot{\varphi} &= -\dfrac{g}{R^2f(\theta)\cos(\theta)}\partial_{\theta}h(\phi, \theta, t) \\
\dot{\theta} &= \dfrac{g}{R^2f(\theta)\cos(\theta)}\partial_{\varphi}h(\phi, \theta, t),
\end{align}

where $ \varphi, \theta $ respectively are the longitudinal and latitudinal position of the particle, $ g $ is the constant of gravity, $ R $ is the mean radius, $ f(\theta) = 2\Omega \sin(\theta) $ is the coriolis parameter and $ \Omega $ is the mean angular velocity of the earth.

The AVISO-data provides global coverage of ocean currents at weekly time-intervals with a spatial resolution of $ 0.25^{\circ} $. As part of this work, we focus on the commonly studied Agulhas region in the Southern Ocean:

\begin{equation}
(\varphi, \theta) \in [14^{\circ}W, 9^{\circ}E] \times [39^{\circ}S, 21^{\circ}S]
\end{equation}

over a period of (maximum) 90 days starting at $ t_0 = 24 $ November 2006. The area under study is characterized by intense eddy activity.

"Nan"-values in the velocity data correspond to points where the particles is not defined (e.g. points on land). At the coastline (=interface between "Nan"-value and "float"-values in the velocity data), the velocity-field needs to be extrapolated, by assuming appropriate boundary conditions. In geophysical flows, the boundary conditions are selected to be of free slip type (=no penetration).
In order to characterize the velocity field at the coastline we interpolate (both in time and space) the velocity from the original resolution to a finer resolution. Whenever the particle is in a grid cell which does not contain "Nan"-values, the velocity field is assumed to be well defined and the velocity can be computed via classic spline interpolation. 
At the coastlines, however, the grid-cell will contain at least one "NaN"-value. Hence, it is not possible to compute the velocity using standard interpolation techniques. In such flow regions, the velocity is computed assuming "free-slip" boundaries.

# Add Folders to Path

We start by adding the necessary folders to the current working path.

In [ ]:
%%time
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-4])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

# add integration folder to current working path
sys.path.append(parent_directory+"/subfunctions/integration")

# Import data

In [2]:
%%time
import scipy.io as sio

#Import velocity data from file in data-folder
mat_file = sio.loadmat('AVISO.mat')

U = mat_file['u']
V = mat_file['v']
x = mat_file['x']
y = mat_file['y']
time_data = mat_file['t']

[[ 0  7 14 21 28 35 42 49 56 63 70 77 84 91]]
Wall time: 2 ms


# Refine velocity data using tricubic spline interpolation from [1]

In order to more accurately capture the velocity data in the open sea we employ the tricubic spline interpolation from [1]. At the coastline, the velocity is evaluated assuming "free-slip" boundary conditions. The velocity data has a spatial resolution of 0.25 degrees, whereas the temporal resolution is originally seven days.

NOTE: The package [pytricubic](#https://github.com/danielguterding/pytricubic) needs to be manually installed. The installation guidelines can be found on [here](#https://github.com/danielguterding/pytricubic).

We refine both the spatial grid spacing to 0.025 degrees and the temporal spacing to one day using a tricubic spline interpolation.

In [4]:
# import tricubic
import tricubic

# import numpy
import numpy as np

# import function which checks location of the particle.
from ipynb.fs.defs.check_location import check_location

# import function which computes the velocity of the particle at the coastline by assuming "free-slip" boundaries.
from ipynb.fs.defs.velocity_boundary import velocity_boundary

# import function which computes the velocity of the particle in a domain where velocity is well defined.
from ipynb.fs.defs.velocity_domain import velocity_domain

X_original, Y_original = np.meshgrid(x, y)
time_data_original = time_data.copy()

Interpolant_x = tricubic.tricubic(list(U), [X_original.shape[0], X_original.shape[1], time_data_original.shape[1]])
Interpolant_y = tricubic.tricubic(list(V), [X_original.shape[0], X_original.shape[1], time_data_original.shape[1]])

X_refined, Y_refined = np.meshgrid(np.arange(np.min(x), np.max(x)+0.025, 0.025), np.arange(np.min(y), np.max(y)+0.025, 0.025)) # 0.025 days
time_refined = np.arange(np.min(time_data), np.max(time_data)+1, 1).reshape(1, -1)  # 1 day resolution

U_refined = np.zeros((X_refined.shape[0], X_refined.shape[1], time_refined.shape[1]))
V_refined = np.zeros((X_refined.shape[0], X_refined.shape[1], time_refined.shape[1]))

for idx_y in range(Y_refined.shape[0]):
    
    for idx_x in range(X_refined.shape[1]):
        
        for idx_t in range(time_refined.shape[1]):
            
            U_refined[idx_y, idx_x, idx_t] = 

# References

[1] Lekien, F., & Marsden, J. (2005). Tricubic interpolation in three dimensions. International Journal for Numerical Methods in Engineering, 63(3), 455-471.